In [1]:
import cv2
import csv  
import numpy as np
import os
import shutil
import keras
import tensorflow.compat.v2 as tf
import matplotlib.pyplot as plt
import tqdm.notebook as tq
from tqdm.auto import tqdm
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPool2D,Dropout,GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils import layer_utils
from keras.engine import training
from keras.utils import data_utils


In [2]:
!pip install -q kaggle
from google.colab import files
files.upload()
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/ 
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [3]:
!kaggle competitions download -c nn23-sports-image-classification

 95% 249M/262M [00:01<00:00, 177MB/s]
100% 262M/262M [00:01<00:00, 158MB/s]


In [4]:
import zipfile

with zipfile.ZipFile('/content/nn23-sports-image-classification.zip','r') as ziphandler:
  ziphandler.extractall('/content/data')

In [5]:
TRAIN_DIR = '/content/data/Train'
TEST_DIR = '/content/data/Test'
IMG_SIZE = 250
LR = 0.0001
batch_size = 64
MODEL_NAME = 'Sports Classification'
CHANNEL_AXIS = -1
TF_WEIGHTS_PATH_NO_TOP = (
    "https://storage.googleapis.com/tensorflow/keras-applications/"
    "xception/xception_weights_tf_dim_ordering_tf_kernels_notop.h5"
)

In [6]:
def return_label(img):
  return img.split('_')[0]

In [7]:

def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        path = os.path.join(TRAIN_DIR, img)
        folder_path = os.path.join(TRAIN_DIR, return_label(img))
        if not os.path.exists(folder_path):
          os.makedirs(folder_path)
        dest = os.path.join(folder_path, img)
        shutil.move(path,dest)
    return training_data


In [8]:
train_data = create_train_data()

  0%|          | 0/1681 [00:00<?, ?it/s]

In [9]:
# Get the data
trdata = ImageDataGenerator(validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)
train_ds = trdata.flow_from_directory(directory="/content/data/Train",target_size=(IMG_SIZE,IMG_SIZE),subset='training')


val_ds = trdata.flow_from_directory(directory="/content/data/Train",target_size=(IMG_SIZE,IMG_SIZE),subset='validation')

Found 1346 images belonging to 6 classes.
Found 335 images belonging to 6 classes.


In [10]:
# xception model
layers = tf.keras.layers
model = Sequential()
img_input = layers.Input(shape=[IMG_SIZE, IMG_SIZE, 3])
 # Block 1
x = layers.Conv2D(
        32, (3, 3), strides=(2, 2), use_bias=False, name="block1_conv1"
    )(img_input)
x = layers.BatchNormalization(axis=CHANNEL_AXIS, name="block1_conv1_bn")(x)
x = layers.Activation("relu", name="block1_conv1_act")(x)
x = layers.Conv2D(64, (3, 3), use_bias=False, name="block1_conv2")(x)
x = layers.BatchNormalization(axis=CHANNEL_AXIS, name="block1_conv2_bn")(x)
x = layers.Activation("relu", name="block1_conv2_act")(x)

residual = layers.Conv2D(
        128, (1, 1), strides=(2, 2), padding="same", use_bias=False
    )(x)
residual = layers.BatchNormalization(axis=CHANNEL_AXIS)(residual)

x = layers.SeparableConv2D(
        128, (3, 3), padding="same", use_bias=False, name="block2_sepconv1"
    )(x)
x = layers.BatchNormalization(axis=CHANNEL_AXIS, name="block2_sepconv1_bn")(
        x
    )
x = layers.Activation("relu", name="block2_sepconv2_act")(x)
x = layers.SeparableConv2D(
        128, (3, 3), padding="same", use_bias=False, name="block2_sepconv2"
    )(x)
x = layers.BatchNormalization(axis=CHANNEL_AXIS, name="block2_sepconv2_bn")(
        x
    )

x = layers.MaxPooling2D(
        (3, 3), strides=(2, 2), padding="same", name="block2_pool"
    )(x)
x = layers.add([x, residual])

residual = layers.Conv2D(
        256, (1, 1), strides=(2, 2), padding="same", use_bias=False
    )(x)
residual = layers.BatchNormalization(axis=CHANNEL_AXIS)(residual)

x = layers.Activation("relu", name="block3_sepconv1_act")(x)
x = layers.SeparableConv2D(
        256, (3, 3), padding="same", use_bias=False, name="block3_sepconv1"
    )(x)
x = layers.BatchNormalization(axis=CHANNEL_AXIS, name="block3_sepconv1_bn")(
        x
    )
x = layers.Activation("relu", name="block3_sepconv2_act")(x)
x = layers.SeparableConv2D(
        256, (3, 3), padding="same", use_bias=False, name="block3_sepconv2"
    )(x)
x = layers.BatchNormalization(axis=CHANNEL_AXIS, name="block3_sepconv2_bn")(
        x
    )

x = layers.MaxPooling2D(
        (3, 3), strides=(2, 2), padding="same", name="block3_pool"
    )(x)
x = layers.add([x, residual])

residual = layers.Conv2D(
        728, (1, 1), strides=(2, 2), padding="same", use_bias=False
    )(x)
residual = layers.BatchNormalization(axis=CHANNEL_AXIS)(residual)

x = layers.Activation("relu", name="block4_sepconv1_act")(x)
x = layers.SeparableConv2D(
        728, (3, 3), padding="same", use_bias=False, name="block4_sepconv1"
    )(x)
x = layers.BatchNormalization(axis=CHANNEL_AXIS, name="block4_sepconv1_bn")(
        x
    )
x = layers.Activation("relu", name="block4_sepconv2_act")(x)
x = layers.SeparableConv2D(
        728, (3, 3), padding="same", use_bias=False, name="block4_sepconv2"
    )(x)
x = layers.BatchNormalization(axis=CHANNEL_AXIS, name="block4_sepconv2_bn")(
        x
    )

x = layers.MaxPooling2D(
        (3, 3), strides=(2, 2), padding="same", name="block4_pool"
    )(x)
x = layers.add([x, residual])

for i in range(8):
  residual = x
  prefix = "block" + str(i + 5)

  x = layers.Activation("relu", name=prefix + "_sepconv1_act")(x)
  x = layers.SeparableConv2D(
            728,
            (3, 3),
            padding="same",
            use_bias=False,
            name=prefix + "_sepconv1",
        )(x)
  x = layers.BatchNormalization(
            axis=CHANNEL_AXIS, name=prefix + "_sepconv1_bn"
        )(x)
  x = layers.Activation("relu", name=prefix + "_sepconv2_act")(x)
  x = layers.SeparableConv2D(
            728,
            (3, 3),
            padding="same",
            use_bias=False,
            name=prefix + "_sepconv2",
        )(x)
  x = layers.BatchNormalization(
            axis=CHANNEL_AXIS, name=prefix + "_sepconv2_bn"
        )(x)
  x = layers.Activation("relu", name=prefix + "_sepconv3_act")(x)
  x = layers.SeparableConv2D(
            728,
            (3, 3),
            padding="same",
            use_bias=False,
            name=prefix + "_sepconv3",
        )(x)
  x = layers.BatchNormalization(
            axis=CHANNEL_AXIS, name=prefix + "_sepconv3_bn"
        )(x)

  x = layers.add([x, residual])

residual = layers.Conv2D(
        1024, (1, 1), strides=(2, 2), padding="same", use_bias=False
    )(x)
residual = layers.BatchNormalization(axis=CHANNEL_AXIS)(residual)

x = layers.Activation("relu", name="block13_sepconv1_act")(x)
x = layers.SeparableConv2D(
        728, (3, 3), padding="same", use_bias=False, name="block13_sepconv1"
    )(x)
x = layers.BatchNormalization(
        axis=CHANNEL_AXIS, name="block13_sepconv1_bn"
    )(x)
x = layers.Activation("relu", name="block13_sepconv2_act")(x)
x = layers.SeparableConv2D(
        1024, (3, 3), padding="same", use_bias=False, name="block13_sepconv2"
    )(x)
x = layers.BatchNormalization(
        axis=CHANNEL_AXIS, name="block13_sepconv2_bn"
    )(x)

x = layers.MaxPooling2D(
        (3, 3), strides=(2, 2), padding="same", name="block13_pool"
    )(x)
x = layers.add([x, residual])

x = layers.SeparableConv2D(
        1536, (3, 3), padding="same", use_bias=False, name="block14_sepconv1"
    )(x)
x = layers.BatchNormalization(
        axis=CHANNEL_AXIS, name="block14_sepconv1_bn"
    )(x)
x = layers.Activation("relu", name="block14_sepconv1_act")(x)

x = layers.SeparableConv2D(
        2048, (3, 3), padding="same", use_bias=False, name="block14_sepconv2"
    )(x)
x = layers.BatchNormalization(
        axis=CHANNEL_AXIS, name="block14_sepconv2_bn"
    )(x)
x = layers.Activation("relu", name="block14_sepconv2_act")(x)

m = training.Model(img_input, x, name="xception")
weights_path = data_utils.get_file(
                "xception_weights_tf_dim_ordering_tf_kernels_notop.h5",
                TF_WEIGHTS_PATH_NO_TOP,
                cache_subdir="models",
                file_hash="b0042744bf5b25fce3cb969f33bebb97",
            )
m.load_weights(weights_path)
model.add(m)

model.add(GlobalAveragePooling2D())
model.add(Flatten())
model.add(Dense(4096,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(6,activation='softmax'))

for layer in model.layers:
  layer.trainable = True
# Add Optimizer and check accuracy metrics
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss=keras.losses.categorical_crossentropy,
              metrics=['accuracy'])
# Check model summary
print(model.summary())

checkpoint = ModelCheckpoint("xception.h5", monitor='val_accuracy', verbose=1, save_best_only=True,
                             save_weights_only=False, mode='auto', period=1)
earlystop = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=20, verbose=1, mode='auto')
hist = model.fit(train_ds, validation_data=val_ds, epochs=100,
                           callbacks=[checkpoint, earlystop])


83683744/83683744 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 8, 8, 2048)        20861480  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 4096)              8392704   
                                                                 
 dropout (Dropout)           (None, 4096)              0         
                                                                 
 dense_1 (Dense)             (None, 6)                 24

None
Epoch 1/100
43/43 [==============================] - ETA: 0s - loss: 0.8056 - accuracy: 0.7303
Epoch 1: val_accuracy improved from -inf to 0.85373, saving model to xception.h5
43/43 [==============================] - 61s 1s/step - loss: 0.8056 - accuracy: 0.7303 - val_loss: 0.4184 - val_accuracy: 0.8537
Epoch 2/100
43/43 [==============================] - ETA: 0s - loss: 0.1590 - accuracy: 0.9450
Epoch 2: val_accuracy improved from 0.85373 to 0.94925, saving model to xception.h5
43/43 [==============================] - 44s 1s/step - loss: 0.1590 - accuracy: 0.9450 - val_loss: 0.1762 - val_accuracy: 0.9493
Epoch 3/100
43/43 [==============================] - ETA: 0s - loss: 0.0763 - accuracy: 0.9792
Epoch 3: val_accuracy improved from 0.94925 to 0.96418, saving model to xception.h5
43/43 [==============================] - 42s 972ms/step - loss: 0.0763 - accuracy: 0.9792 - val_loss: 0.0935 - val_accuracy: 0.9642
Epoch 4/100
43/43 [==============================] - ETA: 0s - loss: 0.

In [ ]:
# vgg16

layers = tf.keras.layers

WEIGHTS_PATH_NO_TOP = (
    "https://storage.googleapis.com/tensorflow/"
    "keras-applications/vgg16/"
    "vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"
)

model = Sequential()
conv_input = layers.Input(shape=[IMG_SIZE, IMG_SIZE, 3])
 # Block 1
x = layers.Conv2D(
        64, (3, 3), activation="relu", padding="same", name="block1_conv1"
    )(conv_input)
x = layers.Conv2D(
        64, (3, 3), activation="relu", padding="same", name="block1_conv2"
    )(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name="block1_pool")(x)

    # Block 2
x = layers.Conv2D(
        128, (3, 3), activation="relu", padding="same", name="block2_conv1"
    )(x)
x = layers.Conv2D(
        128, (3, 3), activation="relu", padding="same", name="block2_conv2"
    )(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name="block2_pool")(x)

    # Block 3
x = layers.Conv2D(
        256, (3, 3), activation="relu", padding="same", name="block3_conv1"
    )(x)
x = layers.Conv2D(
        256, (3, 3), activation="relu", padding="same", name="block3_conv2"
    )(x)
x = layers.Conv2D(
        256, (3, 3), activation="relu", padding="same", name="block3_conv3"
    )(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name="block3_pool")(x)

    # Block 4
x = layers.Conv2D(
        512, (3, 3), activation="relu", padding="same", name="block4_conv1"
    )(x)
x = layers.Conv2D(
        512, (3, 3), activation="relu", padding="same", name="block4_conv2"
    )(x)
x = layers.Conv2D(
        512, (3, 3), activation="relu", padding="same", name="block4_conv3"
    )(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name="block4_pool")(x)

    # Block 5
x = layers.Conv2D(
        512, (3, 3), activation="relu", padding="same", name="block5_conv1"
    )(x)
x = layers.Conv2D(
        512, (3, 3), activation="relu", padding="same", name="block5_conv2"
    )(x)
x = layers.Conv2D(
        512, (3, 3), activation="relu", padding="same", name="block5_conv3"
    )(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name="block5_pool")(x)

m = training.Model(conv_input, x, name="vgg16")
weights_path = data_utils.get_file(
                "vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5",
                WEIGHTS_PATH_NO_TOP,
                cache_subdir="models",
                file_hash="6d6bbae143d832006294945121d1f1fc",
            )
m.load_weights(weights_path)
model.add(m)

model.add(GlobalAveragePooling2D())
model.add(Flatten())
model.add(Dense(4096,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(6,activation='softmax'))

for layer in model.layers:
  layer.trainable = True
# Add Optimizer and check accuracy metrics
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss=keras.losses.categorical_crossentropy,
              metrics=['accuracy'])
# Check model summary
print(model.summary())

checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_accuracy', verbose=1, save_best_only=True,
                             save_weights_only=False, mode='auto', period=1)
earlystop = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=20, verbose=1, mode='auto')
hist = model.fit(train_ds, validation_data=val_ds, epochs=100,
                           callbacks=[checkpoint, earlystop])


Found 1346 images belonging to 6 classes.
Found 335 images belonging to 6 classes.
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 global_average_pooling2d_4   (None, 512)              0         
 (GlobalAveragePooling2D)                                        
                                                                 
 flatten_4 (Flatten)         (None, 512)               0         
                                                                 
 dense_8 (Dense)             (None, 4096)              2101248   
                                                                 
 dropout_4 (Dropout)         (None, 4096)              0         
                                                                 
 dense_9 (Dense)             (None, 6

None
Epoch 1/100
43/43 [==============================] - ETA: 0s - loss: 1.6216 - accuracy: 0.3477
Epoch 1: val_accuracy improved from -inf to 0.55224, saving model to vgg16_1.h5
43/43 [==============================] - 39s 889ms/step - loss: 1.6216 - accuracy: 0.3477 - val_loss: 1.2029 - val_accuracy: 0.5522
Epoch 2/100
43/43 [==============================] - ETA: 0s - loss: 1.3536 - accuracy: 0.4933
Epoch 2: val_accuracy improved from 0.55224 to 0.59701, saving model to vgg16_1.h5
43/43 [==============================] - 36s 845ms/step - loss: 1.3536 - accuracy: 0.4933 - val_loss: 1.0046 - val_accuracy: 0.5970
Epoch 3/100
43/43 [==============================] - ETA: 0s - loss: 1.0157 - accuracy: 0.6174
Epoch 3: val_accuracy improved from 0.59701 to 0.72537, saving model to vgg16_1.h5
43/43 [==============================] - 36s 836ms/step - loss: 1.0157 - accuracy: 0.6174 - val_loss: 0.7192 - val_accuracy: 0.7254
Epoch 4/100
43/43 [==============================] - ETA: 0s - loss:

In [12]:
model_name = 'xception.h5'
saved_model = load_model(model_name, compile = False)
with open('results.csv', 'w', encoding='UTF8',newline='') as f:
  header = ["image_name","label"]
  writer = csv.writer(f)
  writer.writerow(header)
  for img in tq.tqdm(os.listdir(TEST_DIR)):
    path = os.path.join(TEST_DIR, img)
    img_arr = tf.keras.utils.load_img(path,target_size=(IMG_SIZE,IMG_SIZE))
    img_arr = np.asarray(img_arr)
    img_arr = np.expand_dims(img_arr, axis=0)
    prediction = saved_model.predict(img_arr)
    data = [img,np.argmax(prediction)]
    writer.writerow(data)

  

  0%|          | 0/688 [00:00<?, ?it/s]

1/1 [==============================] - 0s 20ms/step
